In [ ]:
######################################################################################
Copyright (c) 2024, Dr. Radhamadhab Dalai,India
Author's email address :  dalai115@gmail.com
#####################################################################################

In [1]:
import random
import math
from collections import defaultdict

# Define Tic-Tac-Toe constants
EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2

# Helper function to check if a player has won the game
def check_winner(board):
    win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                      (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                      (0, 4, 8), (2, 4, 6)]             # Diagonals
    for (i, j, k) in win_conditions:
        if board[i] == board[j] == board[k] and board[i] != EMPTY:
            return board[i]
    if EMPTY not in board:
        return 0  # Tie
    return None  # Game still ongoing

# Helper function to display the Tic-Tac-Toe board
def display_board(board):
    symbols = [' ', 'X', 'O']
    for i in range(0, 9, 3):
        print(f"{symbols[board[i]]} | {symbols[board[i+1]]} | {symbols[board[i+2]]}")
        if i < 6:
            print("--+---+--")

# MCTS Node class that represents a game state
class MCTSNode:
    def __init__(self, board, parent=None, move=None):
        self.board = board          # The current board state
        self.parent = parent        # The parent node (previous state)
        self.children = []          # Child nodes (possible future states)
        self.move = move            # The move that led to this board state
        self.visits = 0             # How many times this node has been visited
        self.wins = 0               # Number of wins resulting from this node

    # Check if this node is fully expanded (i.e., all possible moves have been explored)
    def fully_expanded(self):
        return len(self.children) == len(self.get_possible_moves())

    # Get all possible moves in the current board state
    def get_possible_moves(self):
        return [i for i, val in enumerate(self.board) if val == EMPTY]

    # Expand the node by creating a child node for an unexplored move
    def expand(self, player):
        possible_moves = self.get_possible_moves()
        for move in possible_moves:
            if move not in [child.move for child in self.children]:
                new_board = self.board[:]
                new_board[move] = player
                child_node = MCTSNode(new_board, parent=self, move=move)
                self.children.append(child_node)
                return child_node

    # Choose a child node based on the UCB1 formula
    def best_child(self, exploration_weight=1.4):
        return max(self.children, key=lambda child: child.wins / child.visits +
                   exploration_weight * math.sqrt(math.log(self.visits) / child.visits))

# Monte Carlo Tree Search algorithm
class MCTS:
    def __init__(self, player):
        self.player = player        # The player to run MCTS for (1 for 'X' or 2 for 'O')
        self.wins = defaultdict(int)  # Stores wins for each state
        self.plays = defaultdict(int) # Stores plays for each state

    # Perform a Monte Carlo Tree Search to find the best move
    def search(self, root):
        # 1. Selection: Traverse the tree using the UCB1 formula until we find an unexpanded node
        node = root
        current_player = self.player
        while node.fully_expanded() and node.children:
            node = node.best_child()
            current_player = PLAYER_X if current_player == PLAYER_O else PLAYER_O

        # 2. Expansion: Expand the tree by adding a new child node for the unexplored move
        if not node.fully_expanded():
            node = node.expand(current_player)

        # 3. Simulation: Simulate a random game from this new state
        winner = self.simulate_game(node.board, current_player)

        # 4. Backpropagation: Update the statistics of the tree nodes based on the result
        self.backpropagate(node, winner)

        # Return the best move found from the root node
        return max(root.children, key=lambda child: child.visits).move

    # Simulate a random game from the current board state to a terminal state
    def simulate_game(self, board, player):
        current_board = board[:]
        current_player = player
        while True:
            winner = check_winner(current_board)
            if winner is not None:
                return winner  # Return the winner or tie result (0 for tie)
            possible_moves = [i for i, val in enumerate(current_board) if val == EMPTY]
            move = random.choice(possible_moves)
            current_board[move] = current_player
            current_player = PLAYER_X if current_player == PLAYER_O else PLAYER_O

    # Backpropagate the result of the simulation up the tree
    def backpropagate(self, node, winner):
        while node:
            node.visits += 1
            if winner == self.player:
                node.wins += 1
            elif winner == 0:  # Tie
                node.wins += 0.5
            node = node.parent

# Play Tic-Tac-Toe using MCTS for one player
def play_game():
    board = [EMPTY] * 9  # 3x3 empty Tic-Tac-Toe board
    current_player = PLAYER_X
    mcts_player = MCTS(PLAYER_O)

    while True:
        display_board(board)
        winner = check_winner(board)
        if winner is not None:
            if winner == 0:
                print("It's a tie!")
            else:
                print(f"Player {winner} wins!")
            break

        if current_player == PLAYER_X:
            move = int(input("Enter your move (0-8): "))  # Human player move
        else:
            print("MCTS is thinking...")
            root = MCTSNode(board)
            move = mcts_player.search(root)  # MCTS player move

        board[move] = current_player
        current_player = PLAYER_X if current_player == PLAYER_O else PLAYER_O

# Start the game
if __name__ == "__main__":
    play_game()


  |   |  
--+---+--
  |   |  
--+---+--
  |   |  
Enter your move (0-8): 1
  | X |  
--+---+--
  |   |  
--+---+--
  |   |  
MCTS is thinking...
O | X |  
--+---+--
  |   |  
--+---+--
  |   |  
Enter your move (0-8): 3
O | X |  
--+---+--
X |   |  
--+---+--
  |   |  
MCTS is thinking...
O | X | O
--+---+--
X |   |  
--+---+--
  |   |  
Enter your move (0-8): 5
O | X | O
--+---+--
X |   | X
--+---+--
  |   |  
MCTS is thinking...
O | X | O
--+---+--
X | O | X
--+---+--
  |   |  
Enter your move (0-8): 7
O | X | O
--+---+--
X | O | X
--+---+--
  | X |  
MCTS is thinking...
O | X | O
--+---+--
X | O | X
--+---+--
O | X |  
Player 2 wins!
